In [1]:
import pandas as pd
import numpy as np
import cohere
import re
from tqdm import tqdm
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Paste your API key here. Remember to not share publicly
api_key = open("api_key.txt").readlines()[0]

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

# Preprocessing

In [3]:
df_multilingual = pd.read_csv('data/multilingual.csv')

In [4]:
df_multilingual.head()

,question,answer,url,language,source,section,wrong_answer
0,什么是新型冠状病毒？,新型冠状病毒是一种此前尚未确定的新冠状病毒。导致新型冠状病毒肺炎 | 新冠肺炎2019（CO...,https://chinese.cdc.gov/coronavirus/2019-ncov/...,chinese,CDC,Frequently Asked Questions,使用 CDC 开发的诊断检测，结果为阴性是指在受测者样本中未发现导致 COVID-19 的病...
1,为什么这种疾病被称为 2019 年新型冠状病毒肺炎（COVID-19）？,2020 年 2 月 11 日，世界卫生组织宣布了导致最初于中国武汉发现的 2019 年新型...,https://chinese.cdc.gov/coronavirus/2019-ncov/...,chinese,CDC,Frequently Asked Questions,引发COVID-19的病毒被认为主要以人传人的方式，通过感染患者咳嗽或打喷嚏时产生的呼吸道飞...
2,为什么有人会因为 COVID-19 而指责或躲避某些个人和群体（制造污名）？,美国民众可能对住在或前往COVID-19传播地区的亲友感到担心焦虑。有些人很担心通过这些人感...,https://chinese.cdc.gov/coronavirus/2019-ncov/...,chinese,CDC,Frequently Asked Questions,目前，没有证据表明COVID-19病毒会从宠物的皮肤或皮毛传播给人类。\n关于照顾宠物的最佳...
3,人们如何制止与 COVID-19 相关的污名化？,在注意到污名化正在发生的情况下，人们可以通过提供社会支持来对抗污名化。污名会影响被污名化群体...,https://chinese.cdc.gov/coronavirus/2019-ncov/...,chinese,CDC,Frequently Asked Questions,洗手是保护您与您的家人远离疾病的最佳途径之一。经常用肥皂和水洗手20秒以上，尤其是擤鼻涕、咳...
4,为何某些州的COVID-19病例数量有时与CDC网站上发布的数量有差别？,CDC发布的总病例数量依照各辖区的确认程序得以验证。不同的地方所显示的病例数量的发现和确认程...,https://chinese.cdc.gov/coronavirus/2019-ncov/...,chinese,CDC,Frequently Asked Questions,\n\n与孩子的学校保持联系。\n\n很多学校提供网络课程（虚拟学习）。查看学校布置的作业，...


In [5]:
df_qa = pd.read_csv('data/qa.csv')

In [6]:
df_qa.head()

,AnswerID,Answer
0,ADAM_0003147_Sec1.txt,Question: What is (are) Polycystic ovary syndr...
1,ADAM_0003147_Sec2.txt,Question: What causes Polycystic ovary syndrom...
2,ADAM_0002818_Sec2.txt,Question: What causes Noonan syndrome ?\nURL: ...
3,ADAM_0002818_Sec7.txt,Question: What are the complications of Noonan...
4,ADAM_0002818_Sec9.txt,Question: How to prevent Noonan syndrome ?\nUR...


In [7]:
text = list(df_qa.Answer)

In [8]:
question = []
url = []
answer = []
for i in text:
    x = i.split("\n")
    question.append(x[0].replace("Question: ", ""))
    url.append(x[1].replace("URL: ",""))
    answer.append(" ".join(x[2:]).replace("Answer: ", ""))

In [9]:
for index, row in df_multilingual.iterrows():
    question.append(row['question'])
    url.append(row['url'])
    answer.append(row['answer'])

In [10]:
df_final = pd.DataFrame(zip(question, url, answer), columns =['Question', 'URL', 'Answer'])
df_final.head()

,Question,URL,Answer
0,What is (are) Polycystic ovary syndrome ? (Als...,https://www.nlm.nih.gov/medlineplus/ency/artic...,Polycystic ovary syndrome is a condition in wh...
1,What causes Polycystic ovary syndrome ? (Also ...,https://www.nlm.nih.gov/medlineplus/ency/artic...,PCOS is linked to changes in hormone levels th...
2,What causes Noonan syndrome ?,https://www.nlm.nih.gov/medlineplus/ency/artic...,Noonan syndrome is linked to defects in severa...
3,What are the complications of Noonan syndrome ?,https://www.nlm.nih.gov/medlineplus/ency/artic...,- Buildup of fluid in tissues of body (lymphed...
4,How to prevent Noonan syndrome ?,https://www.nlm.nih.gov/medlineplus/ency/artic...,Couples with a family history of Noonan syndro...


In [15]:
df_final.shape

(3367, 3)

In [25]:
df_final.to_csv("data/finaldf.csv", index=False)

# Training

In [26]:
df = pd.read_csv("data/finaldf.csv")
df.head()

,Question,URL,Answer
0,What is (are) Polycystic ovary syndrome ? (Als...,https://www.nlm.nih.gov/medlineplus/ency/artic...,Polycystic ovary syndrome is a condition in wh...
1,What causes Polycystic ovary syndrome ? (Also ...,https://www.nlm.nih.gov/medlineplus/ency/artic...,PCOS is linked to changes in hormone levels th...
2,What causes Noonan syndrome ?,https://www.nlm.nih.gov/medlineplus/ency/artic...,Noonan syndrome is linked to defects in severa...
3,What are the complications of Noonan syndrome ?,https://www.nlm.nih.gov/medlineplus/ency/artic...,- Buildup of fluid in tissues of body (lymphed...
4,How to prevent Noonan syndrome ?,https://www.nlm.nih.gov/medlineplus/ency/artic...,Couples with a family history of Noonan syndro...


In [ ]:
df_flatten = [' '.join(val[:2048]) for val in df_final.astype(str).values.tolist()]
len(df_flatten)

In [24]:
len(df_flatten[14])

991

In [18]:
response = co.embed(texts=df_flatten, model='multilingual-22-12')  
embeddings = response.embeddings # All text embeddings 
print(embeddings[0][:5])

CohereError: invalid request: texts[12] is too long. Max tokens per text must be at most 2048 - received 2358. Text: 'What causes Chronic Diarrhea in Children ? http://www.niddk.nih.gov/health-information/health-topics/digestive-diseases/diarrhea/Pages/diarrhea-in-children-facts.aspx Many diseases and disorders can cause chronic diarrhea in children. Common causes include                  - infections  - functional gastrointestinal (GI) disorders  - food allergies and intolerances  - inflammatory bowel disease (IBD)                  Infections, food allergies and intolerances, and IBD may cause chronic diarrhea along with malabsorption, meaning the small intestine does not absorb nutrients from food. If children do not absorb enough nutrients from the food they eat, they may become malnourished. Functional GI disorders do not cause malabsorption.                  Infections                  Infections from viruses, bacteria, or parasites sometimes lead to chronic diarrhea. After an inf

In [ ]:
# Check the dimensions of the embeddings
embeds = np.array(embeds)
embeds.shape

In [9]:
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')

True

In [10]:
# Choose an example (we'll retrieve others similar to it)
example_id = 78

# Retrieve nearest neighbors
similar_item_ids = search_index.get_nns_by_item(example_id,10,
                                                include_distances=True)
# Format and print the text and distances
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['question'], 
                             'distance': similar_item_ids[1]}).drop(example_id)

print(f"Question:'{df.iloc[example_id]['question']}'\nNearest neighbors:")
results

Question:'Why is the disease being called coronavirus disease 2019, COVID-19?'
Nearest neighbors:


,texts,distance
559,What is COVID-19?,0.708529
232,¿Por qué a la enfermedad la llaman enfermedad ...,0.813518
77,What is a novel coronavirus?,0.813775
558,What is a coronavirus?,0.842466
569,Is COVID-19 the same as SARS?,0.852085
1,为什么这种疾病被称为 2019 年新型冠状病毒肺炎（COVID-19）？,0.872134
578,Is the source of the coronavirus causing COVID...,0.923686
473,什么是2019冠状病毒病？,0.951086
804,¿Qué es la COVID-19?,0.960803


In [11]:
query = "what ages are more prone to die from covid"

# Get the query's embedding
query_embed = co.embed(texts=[query],
                  model="large",
                  truncate="LEFT").embeddings

# Retrieve the nearest neighbors
similar_item_ids = search_index.get_nns_by_vector(query_embed[0],3,
                                                include_distances=True)
# Format the results
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['question'], 
                             'distance': similar_item_ids[1]})


print(f"Query:'{query}'\nNearest neighbors:")
results

Query:'what ages are more prone to die from covid'
Nearest neighbors:


,texts,distance
131,Who is at higher risk for serious illness from...,0.724839
133,How were the underlying conditions for people ...,0.951346
565,Who is at risk of developing severe illness?,0.977805
